In [2]:
import pybullet as b
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa.urdf"

def get_rot(T):
    R = e.Matrix3d()
    for row in range(R.rows()):
        for col in range(R.cols()):
            R.coeff(row, col, T.coeff(row, col))
    return R

def get_pos(T):
    p = e.Vector3d()
    for i in range(p.rows()):
        p[i] = T.coeff(i,3)
    return p

def get_col(R, c=0):
    assert c <= R.cols(), "column index out of bounds"
    assert R.rows() == R.cols(), "square matrix expected"
    assert R.rows() == 3, "3x3 matrix expected"
    v = e.Vector3d()
    for i in range(R.rows()):
        v[i] = R.coeff(i,c)
    return v

def get_orientation_error(R, R_d):
    '''Calculates orientation error
    @param R Current orientation in Mat3d
    @param R_d Desired orientation in Mat3d
    @return del_phi "R_d-R" error
    '''
    c1 = get_col(R,0)
    c2 = get_col(R,1)
    c3 = get_col(R,2)
    d1 = get_col(R_d,0)
    d2 = get_col(R_d,1)
    d3 = get_col(R_d,2)
    
    return 1.0/2.0*(c1.cross(d1)+c2.cross(d2)+c3.cross(d3))
    
    
def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation();
    
    for row in range(R.rows()):
        for col in range(R.cols()):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(p.rows()):
        m4d.coeff(row, 3, p[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))

# import robot as global variable
p = from_urdf_file(URDF)
print("Imported " + p.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,0]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF(URDF,startPos, startOrientation, flags=loadFlag)

# [note] assume the last joint is a fixed joint
nDof = b.getNumJoints(robotId)
nDofActuated = nDof-1
jointId = range(nDofActuated)
toolJointId = jointId[-1]+1

# gains
Kp1 = e.MatrixXd(np.diag([10000,10000,10000,10000,10000,10000]))
Kd1 = e.MatrixXd(np.diag([200,200,200,100,100,100]))
Kp2 = 10000
Kd2 = 200

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1) # linear
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1) # angular

# task definitions
bodyName1 = "lbr_iiwa_link_7"
S1 = S

# setup configuration
b.setJointMotorControlArray(robotId, jointId, b.VELOCITY_CONTROL, forces=[0] * nDofActuated)
q_home = np.zeros(nDofActuated)
for i in jointId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
q_home = e.VectorXd(q_home)

b.enableJointForceTorqueSensor(robotId, toolJointId-1, enableSensor=1)
b.enableJointForceTorqueSensor(robotId, toolJointId, enableSensor=1)

print("toolID is {}".format(toolJointId))

ImportError: libtinyxml2.so.8: cannot open shared object file: No such file or directory

In [ ]:
# simulate torque control
loop_counter = 0
for i in range(1000000):
    loop_counter += 1
    joint_states = b.getJointStates(robotId, jointId)
    
    # update q and dq states from simulation
    q = [joint_states[i][0] for i in jointId]
    dq = [joint_states[i][1] for i in jointId]
    for i in range(0, nDofActuated):
        p.mbc.q[i+1][0] = q[i]
        p.mbc.alpha[i+1][0] = dq[i]

    # forward kinematics
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q)
    dq = e.VectorXd(dq)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(0,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName1, p.mb.bodies())])

    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName1.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ1 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ1);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ1);
    J1 = S*tempJ1
    dJ1 = S*tempdJ1
    
    # jacobian, dJacobian [angular; linear]
    J2 = e.MatrixXd(1,nDofActuated)
    J2.coeff(0,0,1)
    dJ2 = e.MatrixXd(1,nDofActuated)
    
    # mass matrix
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # compute Minv
    Minv = M.inverse()
    
    # compute lambda1, Jbar1, Nprec1
    lambda1 = inv_spd_sr(J1*Minv*J1.transpose(),1e-3)
    J1bar = Minv*J1.transpose()*lambda1
    Nprec1 = e.MatrixXd.Identity(nDof, nDof) - J1bar*J1
    
    # compute lambda21, Jbar21
    J21 = J2*Nprec1
    lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-3)
    J21bar = Minv*J21.transpose()*lambda21
    Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
    # error, task1
    err1 = e.VectorXd(S.rows())
    x1_des = e.Vector3d(0.5,0,0.5)
    R1_des = e.MatrixXd(3,3)
    R1_des.coeff(1,0,1)
    R1_des.coeff(2,1,1)
    R1_des.coeff(0,2,1)
    p1 = get_pos(T1)
    R1 = get_rot(T1)
    err1_pos = x1_des - p1
    err1_rot = get_orientation_error(R1, R1_des)
    for i in range(err1_pos.rows()):
        err1[i] = err1_pos[i]
        err1[i+err1_pos.rows()] = err1_rot[i]
    # error, task2
    err2 = e.VectorXd([0-q[0]])
    # derror, task1
    derr1 = -J1*dq
    # derror, task2
    derr2 = -J2*dq
    
    # FT states
    tool_state = b.getJointState(robotId, 7)
    reaction_force = tool_state[2]
    if((loop_counter % 200) == 0):
        print(reaction_force)
    
    # control
    tau_1 = e.VectorXd(J1.transpose()*lambda1*(Kp1*err1+Kd1*derr1-dJ1*dq))
    tau_2 = e.VectorXd(J21.transpose()*lambda21*(Kp2*err2+Kd2*derr2-dJ2*dq-J2*Minv*tau_1))
    tau = tau_1 + Nprec1.transpose()*tau_2 + h
    
    # display
#     b.addUserDebugLine(
#         lineFromXYZ = [0, 0, 0],
#         lineToXYZ = [0.3, 0, 0],
#         lineColorRGB = [1, 0, 0],
#         lineWidth = 2,
#         lifeTime = 0.1,
#         parentObjectUniqueId = robotId,
#         parentLinkIndex = 7)
#     b.addUserDebugLine(
#         lineFromXYZ = [0, 0, 0],
#         lineToXYZ = [0, 0.3, 0],
#         lineColorRGB = [0, 1, 0],
#         lineWidth = 2,
#         lifeTime = 0.1,
#         parentObjectUniqueId = robotId,
#         parentLinkIndex = 7)
#     b.addUserDebugLine(
#         lineFromXYZ = [0, 0, 0],
#         lineToXYZ = [0, 0, 0.3],
#         lineColorRGB = [0, 0, 1],
#         lineWidth = 2,
#         lifeTime = 0.1,
#         parentObjectUniqueId = robotId,
#         parentLinkIndex = 7)
    
    # command system:
    for i in jointId:
        b.setJointMotorControl2(robotId, i, b.TORQUE_CONTROL, force=tau[i])
    
    b.stepSimulation()
    time.sleep(DT)
b.disconnect()

(-3.920554451788285, 21.529430123397177, 2.207663905436936, -5.046352103416509e-06, -4.102659000110746e-06, -4.707577834749311e-05)
(0.16097560298123242, 19.639789625644674, 2.1815379529650114, -1.2314366174098444e-06, 6.639693753449178e-07, -1.7906115917366163e-06)
(0.22255420018055666, 19.639036861073336, 2.181485394017513, -1.229209160635615e-06, 6.776289921922952e-07, -6.018483365583676e-08)
(0.22462455899335265, 19.6390089886789, 2.1814836297357068, -1.2291334693635997e-06, 6.780896826075648e-07, -2.0226869939797245e-09)
(0.22469413905465513, 19.639008046704312, 2.181483570430426, -1.2291309199694566e-06, 6.781051652433078e-07, -6.797799006812795e-11)
(0.2246964774846193, 19.639008015043107, 2.1814835684342504, -1.2291308342878354e-06, 6.78105685579831e-07, -2.2845871200305784e-12)
(0.22469655607402705, 19.639008013967896, 2.1814835683687677, -1.2291308313906393e-06, 6.781057030672348e-07, -7.677508647797056e-14)
(0.22469655871522248, 19.63900801393894, 2.181483568365981, -1.22913